<a href="https://colab.research.google.com/github/Ftajedini/Github_Tutorial/blob/main/Copy_of_DashbordDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Building a change detection app using Jupyter Dashboard

In [ ]:
!pip install arcgis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pandas-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Using cached pylerc-4.0-py3-none-any.whl
  Using cached ujson-5.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (53 kB)
  Using cached jupyterlab-4.0.9-py3-none-any.whl (9.2 MB)
  Using cached geomet-1.1.0-py3-none-any.whl (31 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Using cached pyspnego-0.10.2-py3-none-any.whl (129 kB)
  Using cached requests_kerberos-0.14.0-py2.py3-none-any.whl (11 kB)
  Using cached requests_gssapi-1.2.3-py2.py3-none-any.whl
  Using cached truststore-0.8.0-py3-none-any.whl (16 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached async_lru-2.0.4-py3-none-any.whl (6.1 kB)
  Using cached jupyter_lsp-2.2.0-py3-none-any.whl (65 kB)
  Using cached jupyter_server-2.10.1-py3-none-any.

In [ ]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from arcgis.raster.functions import *
from arcgis import geometry
import pandas as pd
import ipywidgets as widgets


In [ ]:
gis = GIS()

In [ ]:
# search for the landsat multispectral imagery layer
landsat_item = gis.content.search("Landsat Multispectral tags:'Landsat on AWS','landsat 8', 'Multispectral', 'Multitemporal', 'imagery', 'temporal', 'MS'", 'Imagery Layer', outside_org=True)[0]
landsat = landsat_item.layers[0]
df = None

##Create widget controls to accept place of interest
We use the widgets module from ipywidgets to create a text box and command button controls. These controls allow the user to specify a place of interest.

In [ ]:
# text box widget
location = widgets.Text(value='Ranchi, India', placeholder='Ranchi, India',
                        description='Location:', disabled=False)

In [ ]:
# command button widget
gobtn = widgets.Button(description='Go', disabled=False,
                       button_style='', tooltip='Go', icon='check')


In [ ]:
# define what happens whent the command button is clicked
def on_gobutton_clicked(b):
    global df
    global m
    global oldslider

    # geocode the place name and set that as the map's extent
    area = geocode(location.value)[0]
    m.extent = area['extent']
    df = filter_images()
gobtn.on_click(on_gobutton_clicked)
location_items = [location, gobtn]
widgets.HBox(location_items)

In [ ]:
m = gis.map(location.value)
m.add_layer(landsat)
display(m)

MapView(layout=Layout(height='400px', width='100%'))

##Create date slider controls
We create two slider controls to pick the before and after dates.

In [ ]:
oldindex =  0 # int(len(df)/2)

# before image date slider
oldslider = widgets.IntSlider(value=oldindex, min=0,max=10, #len(df) - 1,
                              step=1, description='Older:', disabled=False,
                              continuous_update=True, orientation='horizontal',
                              readout=False, readout_format='f', slider_color='white')

old_label = widgets.Label(value='')#str(df.Time.iloc[oldindex].date()))

# define the slider behavior
def on_old_value_change(change):
    global df
    i = change['new']
    if df is not None:
        try:
            # print(df.Time.iloc[i].date())
            old_label.value = str(df.Time.iloc[i].date())
        except:
            pass

oldslider.observe(on_old_value_change, names='value')
widgets.HBox([oldslider, old_label])

In [ ]:
newindex = 0 # len(df) - 1

# after image date slider
newslider = widgets.IntSlider(value=newindex, min=0, max=10, #len(df) - 1,
                              step=1, description='Newer:', disabled=False,
                              continuous_update=True, orientation='horizontal',
                              readout=False, readout_format='f', slider_color='white')

new_label = widgets.Label(value='') #str(df.Time.iloc[newindex].date()))

# define the slider behavior
def on_new_value_change(change):
    global df
    i = change['new']
    if df is not None:
        try:
        # print(df.Time.iloc[i].date())
            new_label.value = str(df.Time.iloc[i].date())
        except:
            pass
newslider.observe(on_new_value_change, names='value')
widgets.HBox([newslider, new_label])

##Query the time enabled landast imagery layer
Based on the dates selected from the sliders, we filter the layer for images

In [ ]:
def update_sliders(tdf):
    global oldslider
    global newslider

    oldslider.max = len(tdf) - 1
    newslider.max = len(tdf) -1
    oldindex = int(len(tdf)/2)
    newindex = int(len(tdf) -1)
    oldslider.value = oldindex
    newslider.value = newindex
    old_label.value = str(tdf.Time.iloc[oldindex].date())
    new_label.value = str(tdf.Time.iloc[newindex].date())


def filter_images():
    global df
    area = geocode(location.value, out_sr=landsat.properties.spatialReference)[0]
    extent = area['extent']

    selected = landsat.filter_by(where="(Category = 1) AND (CloudCover <=0.10)",
                             geometry=geometry.filters.intersects(extent))
    fs = selected.query(out_fields="AcquisitionDate, GroupName, Best, CloudCover, WRS_Row, WRS_Path, Month, Name",
                  return_geometry=True,
                  return_distinct_values=False,
                  order_by_fields="AcquisitionDate")
    tdf = fs.sdf
    df = tdf
    tdf['Time'] = pd.to_datetime(tdf['AcquisitionDate'], unit='ms')

    if len(tdf) > 1:
        update_sliders(tdf)

    # m.draw(tdf.iloc[oldslider.value].SHAPE)

    return tdf

In [ ]:
df = filter_images()

In [ ]:
df

,OBJECTID,AcquisitionDate,GroupName,Best,CloudCover,WRS_Row,WRS_Path,Month,Name,SHAPE,Time
0,3109245,1972-11-07 00:00:00,p150r044_1x19721107,95850044,-0.01,44,150,11,p150r044_1dm19721107_z45_MS,"{""rings"": [[[9725537.754999999, 2704838.241700...",1972-11-07 00:00:00
1,3109280,1975-12-08 00:00:00,p151r044_2x19751208,95849044,-0.01,44,151,12,p151r044_2dm19751208_z45_MS,"{""rings"": [[[9571542.173799999, 2737083.279300...",1975-12-08 00:00:00
2,3101039,1992-11-01 00:00:00,p140r044_5x19921101,91860044,-0.01,44,140,11,p140r044_5dt19921101_z45_MS,"{""rings"": [[[9656510.8985, 2731066.4963000007]...",1992-11-01 00:00:00
3,3092777,2001-11-02 00:00:00,p140r044_7x20011102,88860044,0.0,44,140,11,p140r044_7dt20011102_z45_MS,"{""rings"": [[[9660501.5825, 2724625.0896999985]...",2001-11-02 00:00:00
4,3078638,2005-11-05 00:00:00,L5140044_04420051105,79860044,0.0,44,140,11,L5140044_04420051105_MS,"{""rings"": [[[9668660.803, 2720957.0716999993],...",2005-11-05 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
167,4535387,2023-06-08 04:42:47,LC09_L1TP_140044_20230608_20230608_02_T1_MTL,2860044,0.0012,44,140,6,LC09_L1TP_140044_20230608_20230608_02_T1,"{""rings"": [[[9661059.686099999, 2726107.541000...",2023-06-08 04:42:47
168,4784211,2023-10-14 04:43:35,LC09_L1TP_140044_20231014_20231014_02_T1_MTL,1860044,0.0005,44,140,10,LC09_L1TP_140044_20231014_20231014_02_T1,"{""rings"": [[[9663554.693300001, 2725962.322299...",2023-10-14 04:43:35
169,4803970,2023-10-30 04:43:32,LC09_L1TP_140044_20231030_20231030_02_T1_MTL,860044,0.0041,44,140,10,LC09_L1TP_140044_20231030_20231030_02_T1,"{""rings"": [[[9666049.850000001, 2726091.738000...",2023-10-30 04:43:32
170,4820303,2023-11-06 04:49:42,LC09_L1TP_141044_20231106_20231106_02_T1_MTL,859044,0.0,44,141,11,LC09_L1TP_141044_20231106_20231106_02_T1,"{""rings"": [[[9494864.395399999, 2726065.998399...",2023-11-06 04:49:42


##Perform change detection when the action button is clicked
We create a command button and when it is clicked, display the difference in NDVI values in shades of green and magenta.

In [ ]:
# create the action button
diffbtn = widgets.Button(description='Detect changes', disabled=False,
                         button_style='success', tooltip='Show Different Image',
                         icon='check')

def on_diffbutton_clicked(b):
    # m.clear_graphics()
    first = df.iloc[oldslider.value].OBJECTID
    last = df.iloc[newslider.value].OBJECTID
    old = landsat.filter_by('OBJECTID='+str(first))
    new = landsat.filter_by('OBJECTID='+str(last))
    diff = stretch(composite_band([ndvi(old, '5 4'),
                               ndvi(new, '5 4'),
                               ndvi(old, '5 4')]),
                               stretch_type='stddev',  num_stddev=3, min=0, max=255, dra=True, astype='u8')
    m.add_layer(diff)

diffbtn.on_click(on_diffbutton_clicked)
diffbtn

Button(button_style='success', description='Detect changes', icon='check', style=ButtonStyle(), tooltip='Show …